In [ ]:
!pip install pretrainedmodels torchsummary opencv-python

In [ ]:
import dlib
dlib.__version__

In [ ]:
import json
from torchvision import transforms
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.model_zoo as model_zoo
from torch.nn import init
import os
import argparse
import pretrainedmodels
import torchvision
from torchsummary import summary
from keras.models import load_model
from os.path import join
import cv2
import dlib
from PIL import Image as pil_image
from tqdm import tqdm
import numpy as np
import time

In [ ]:
xception_default_data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((299, 299)),
        transforms.ToTensor(),
        transforms.Normalize([0.5]*3, [0.5]*3)
    ]),
    'val': transforms.Compose([
        transforms.Resize((299, 299)),
        transforms.ToTensor(),
        transforms.Normalize([0.5] * 3, [0.5] * 3)
    ]),
    'test': transforms.Compose([
        transforms.Resize((299, 299)),
        transforms.ToTensor(),
        transforms.Normalize([0.5] * 3, [0.5] * 3)
    ]),
}

In [ ]:
pretrained_settings = {
    'xception': {
        'imagenet': {
            'url': 'http://data.lip6.fr/cadene/pretrainedmodels/xception-b5690688.pth',
            'input_space': 'RGB',
            'input_size': [3, 299, 299],
            'input_range': [0, 1],
            'mean': [0.5, 0.5, 0.5],
            'std': [0.5, 0.5, 0.5],
            'num_classes': 1000,
            'scale': 0.8975 # The resize parameter of the validation transform should be 333, and make sure to center crop at 299x299
        }
    }
}

In [ ]:
class SeparableConv2d(nn.Module):
    def __init__(self,in_channels,out_channels,kernel_size=1,stride=1,padding=0,dilation=1,bias=False):
        super(SeparableConv2d,self).__init__()

        self.conv1 = nn.Conv2d(in_channels,in_channels,kernel_size,stride,padding,dilation,groups=in_channels,bias=bias)
        self.pointwise = nn.Conv2d(in_channels,out_channels,1,1,0,1,1,bias=bias)

    def forward(self,x):
        x = self.conv1(x)
        x = self.pointwise(x)
        return x

In [ ]:
class Block(nn.Module):
    def __init__(self,in_filters,out_filters,reps,strides=1,start_with_relu=True,grow_first=True):
        super(Block, self).__init__()

        if out_filters != in_filters or strides!=1:
            self.skip = nn.Conv2d(in_filters,out_filters,1,stride=strides, bias=False)
            self.skipbn = nn.BatchNorm2d(out_filters)
        else:
            self.skip=None

        self.relu = nn.ReLU(inplace=True)
        rep=[]

        filters=in_filters
        if grow_first:
            rep.append(self.relu)
            rep.append(SeparableConv2d(in_filters,out_filters,3,stride=1,padding=1,bias=False))
            rep.append(nn.BatchNorm2d(out_filters))
            filters = out_filters

        for i in range(reps-1):
            rep.append(self.relu)
            rep.append(SeparableConv2d(filters,filters,3,stride=1,padding=1,bias=False))
            rep.append(nn.BatchNorm2d(filters))

        if not grow_first:
            rep.append(self.relu)
            rep.append(SeparableConv2d(in_filters,out_filters,3,stride=1,padding=1,bias=False))
            rep.append(nn.BatchNorm2d(out_filters))

        if not start_with_relu:
            rep = rep[1:]
        else:
            rep[0] = nn.ReLU(inplace=False)

        if strides != 1:
            rep.append(nn.MaxPool2d(3,strides,1))
        self.rep = nn.Sequential(*rep)

    def forward(self,inp):
        x = self.rep(inp)

        if self.skip is not None:
            skip = self.skip(inp)
            skip = self.skipbn(skip)
        else:
            skip = inp

        x+=skip
        return x

In [ ]:
class Xception(nn.Module):
    """
    Xception optimized for the ImageNet dataset, as specified in
    https://arxiv.org/pdf/1610.02357.pdf
    """
    def __init__(self, num_classes=1000):
        """ Constructor
        Args:
            num_classes: number of classes
        """
        super(Xception, self).__init__()
        self.num_classes = num_classes

        self.conv1 = nn.Conv2d(3, 32, 3,2, 0, bias=False)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu = nn.ReLU(inplace=True)

        self.conv2 = nn.Conv2d(32,64,3,bias=False)
        self.bn2 = nn.BatchNorm2d(64)
        #do relu here

        self.block1=Block(64,128,2,2,start_with_relu=False,grow_first=True)
        self.block2=Block(128,256,2,2,start_with_relu=True,grow_first=True)
        self.block3=Block(256,728,2,2,start_with_relu=True,grow_first=True)

        self.block4=Block(728,728,3,1,start_with_relu=True,grow_first=True)
        self.block5=Block(728,728,3,1,start_with_relu=True,grow_first=True)
        self.block6=Block(728,728,3,1,start_with_relu=True,grow_first=True)
        self.block7=Block(728,728,3,1,start_with_relu=True,grow_first=True)

        self.block8=Block(728,728,3,1,start_with_relu=True,grow_first=True)
        self.block9=Block(728,728,3,1,start_with_relu=True,grow_first=True)
        self.block10=Block(728,728,3,1,start_with_relu=True,grow_first=True)
        self.block11=Block(728,728,3,1,start_with_relu=True,grow_first=True)

        self.block12=Block(728,1024,2,2,start_with_relu=True,grow_first=False)

        self.conv3 = SeparableConv2d(1024,1536,3,1,1)
        self.bn3 = nn.BatchNorm2d(1536)

        #do relu here
        self.conv4 = SeparableConv2d(1536,2048,3,1,1)
        self.bn4 = nn.BatchNorm2d(2048)

        self.fc = nn.Linear(2048, num_classes)

    def features(self, input):
        x = self.conv1(input)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)

        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        x = self.block6(x)
        x = self.block7(x)
        x = self.block8(x)
        x = self.block9(x)
        x = self.block10(x)
        x = self.block11(x)
        x = self.block12(x)

        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu(x)

        x = self.conv4(x)
        x = self.bn4(x)
        return x

    def logits(self, features):
        x = self.relu(features)

        x = F.adaptive_avg_pool2d(x, (1, 1))
        x = x.view(x.size(0), -1)
        x = self.last_linear(x)
        return x

    def forward(self, input):
        x = self.features(input)
        x = self.logits(x)
        return x

In [ ]:
def xception(num_classes=1000, pretrained='imagenet'):
    model = Xception(num_classes=num_classes)
    if pretrained:
        settings = pretrained_settings['xception'][pretrained]
        assert num_classes == settings['num_classes'], \
            "num_classes should be {}, but is {}".format(settings['num_classes'], num_classes)

        model = Xception(num_classes=num_classes)
        model.load_state_dict(model_zoo.load_url(settings['url']))

        model.input_space = settings['input_space']
        model.input_size = settings['input_size']
        model.input_range = settings['input_range']
        model.mean = settings['mean']
        model.std = settings['std']

    # TODO: ugly
    model.last_linear = model.fc
    del model.fc
    return model

In [ ]:
def return_pytorch04_xception(pretrained=True):
    # Raises warning "src not broadcastable to dst" but thats fine
    model = xception(pretrained=False)
    if pretrained:
        # Load model in torch 0.4+
        model.fc = model.last_linear
        del model.last_linear
        # path = os.path.join(os.path.dirname(__file__), "sagemaker-studiolab-notebooks/xception-b5690688.pth")
        path = "xception-b5690688.pth"
        state_dict = torch.load(path)
        for name, weights in state_dict.items():
            if 'pointwise' in name:
                state_dict[name] = weights.unsqueeze(-1).unsqueeze(-1)
        model.load_state_dict(state_dict)
        model.last_linear = model.fc
        del model.fc
    return model

In [ ]:
class TransferModel(nn.Module):
    """
    Simple transfer learning model that takes an imagenet pretrained model with
    a fc layer as base model and retrains a new fc layer for num_out_classes
    """
    def __init__(self, modelchoice, num_out_classes=2, dropout=0.0):
        super(TransferModel, self).__init__()
        self.modelchoice = modelchoice
        if modelchoice == 'xception':
            self.model = return_pytorch04_xception()
            # Replace fc
            num_ftrs = self.model.last_linear.in_features
            if not dropout:
                self.model.last_linear = nn.Linear(num_ftrs, num_out_classes)
            else:
                print('Using dropout', dropout)
                self.model.last_linear = nn.Sequential(
                    nn.Dropout(p=dropout),
                    nn.Linear(num_ftrs, num_out_classes)
                )
        elif modelchoice == 'resnet50' or modelchoice == 'resnet18':
            if modelchoice == 'resnet50':
                self.model = torchvision.models.resnet50(pretrained=True)
            if modelchoice == 'resnet18':
                self.model = torchvision.models.resnet18(pretrained=True)
            # Replace fc
            num_ftrs = self.model.fc.in_features
            if not dropout:
                self.model.fc = nn.Linear(num_ftrs, num_out_classes)
            else:
                self.model.fc = nn.Sequential(
                    nn.Dropout(p=dropout),
                    nn.Linear(num_ftrs, num_out_classes)
                )
        elif modelchoice == 'densenet121':
            self.model = torchvision.models.densenet121(pretrained=True)
            # Replace fc
            num_ftrs = self.model.classifier.in_features
            if not dropout:
                self.model.classifier = nn.Linear(num_ftrs, num_out_classes)
            else:
                self.model.classifier = nn.Sequential(
                    nn.Dropout(p=dropout),
                    nn.Linear(num_ftrs, num_out_classes)
                )
        else:
            raise Exception('Choose valid model, e.g. resnet50')

    def set_trainable_up_to(self, boolean, layername="Conv2d_4a_3x3"):
        """
        Freezes all layers below a specific layer and sets the following layers
        to true if boolean else only the fully connected final layer
        :param boolean:
        :param layername: depends on network, for inception e.g. Conv2d_4a_3x3
        :return:
        """
        # Stage-1: freeze all the layers
        if layername is None:
            for i, param in self.model.named_parameters():
                param.requires_grad = True
                return
        else:
            for i, param in self.model.named_parameters():
                param.requires_grad = False
        if boolean:
            # Make all layers following the layername layer trainable
            ct = []
            found = False
            for name, child in self.model.named_children():
                if layername in ct:
                    found = True
                    for params in child.parameters():
                        params.requires_grad = True
                ct.append(name)
            if not found:
                raise Exception('Layer not found, cant finetune!'.format(
                    layername))
        else:
            if self.modelchoice == 'xception':
                # Make fc trainable
                for param in self.model.last_linear.parameters():
                    param.requires_grad = True

            elif self.modelchoice == 'resnet50' or self.modelchoice == 'resnet18':
                # Make fc trainable
                for param in self.model.fc.parameters():
                    param.requires_grad = True

            elif self.modelchoice == 'densenet121':
                # Make fc trainable
                for param in self.model.classifier.parameters():
                    param.requires_grad = True

            else:
                # Make fc trainable
                for param in self.model.fc.parameters():
                    param.requires_grad = True

    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
def model_selection(modelname, num_out_classes,
                    dropout=None):
    """
    :param modelname:
    :return: model, image size, pretraining<yes/no>, input_list
    """
    if modelname == 'xception':
        return TransferModel(modelchoice='xception',
                             num_out_classes=num_out_classes), 299, \
               True, ['image'], None
    elif modelname == 'resnet18':
        return TransferModel(modelchoice='resnet18', dropout=dropout,
                             num_out_classes=num_out_classes), \
               224, True, ['image'], None
    elif modelname == 'densenet121':
        return TransferModel(modelchoice='densenet121', dropout=dropout,
                             num_out_classes=num_out_classes), \
               224, True, ['image'], None
    else:
        raise NotImplementedError(modelname)

In [ ]:
def get_boundingbox(face, width, height, scale=1.3, minsize=None):
    """
    Expects a dlib face to generate a quadratic bounding box.
    :param face: dlib face class
    :param width: frame width
    :param height: frame height
    :param scale: bounding box size multiplier to get a bigger face region
    :param minsize: set minimum bounding box size
    :return: x, y, bounding_box_size in opencv form
    """
    x1 = face.left()
    y1 = face.top()
    x2 = face.right()
    y2 = face.bottom()
    size_bb = int(max(x2 - x1, y2 - y1) * scale)
    if minsize:
        if size_bb < minsize:
            size_bb = minsize
    center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2

    # Check for out of bounds, x-y top left corner
    x1 = max(int(center_x - size_bb // 2), 0)
    y1 = max(int(center_y - size_bb // 2), 0)
    # Check for too big bb size for given x, y
    size_bb = min(width - x1, size_bb)
    size_bb = min(height - y1, size_bb)

    return x1, y1, size_bb

In [ ]:
def preprocess_image(image, cuda=True):
    """
    Preprocesses the image such that it can be fed into our network.
    During this process we envoke PIL to cast it into a PIL image.

    :param image: numpy image in opencv form (i.e., BGR and of shape
    :return: pytorch tensor of shape [1, 3, image_size, image_size], not
    necessarily casted to cuda
    """
    # Revert from BGR
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Preprocess using the preprocessing function used during training and
    # casting it to PIL image
    preprocess = xception_default_data_transforms['test']
    preprocessed_image = preprocess(pil_image.fromarray(image))
    # Add first dimension as the network expects a batch
    preprocessed_image = preprocessed_image.unsqueeze(0)
    if cuda:
        preprocessed_image = preprocessed_image.cuda()
    return preprocessed_image

In [ ]:
def predict_with_model(image, model, post_function=nn.Softmax(dim=1),
                       cuda=True):
    """
    Predicts the label of an input image. Preprocesses the input image and
    casts it to cuda if required

    :param image: numpy image
    :param model: torch model with linear layer at the end
    :param post_function: e.g., softmax
    :param cuda: enables cuda, must be the same parameter as the model
    :return: prediction (1 = fake, 0 = real)
    """
    # Preprocess
    preprocessed_image = preprocess_image(image, cuda)

    # Model prediction
    output = model(preprocessed_image)
    output = post_function(output)

    # Cast to desired
    _, prediction = torch.max(output, 1)    # argmax
    prediction = float(prediction.cpu().numpy())

    return int(prediction), output

In [ ]:
def test_full_image_network(video_path, model_path=None,
                            start_frame=0, end_frame=None, cuda=True):
    """
    Reads a video and evaluates a subset of frames with the a detection network
    that takes in a full frame. Outputs are only given if a face is present
    and the face is highlighted using dlib.
    :param video_path: path to video file
    :param model_path: path to model file (should expect the full sized image)
    :param start_frame: first frame to evaluate
    :param end_frame: last frame to evaluate
    :param cuda: enable cuda
    :return:
    """
    print('Starting: {}'.format(video_path))

    # Read and write
    reader = cv2.VideoCapture(video_path)
    num_frames = int(reader.get(cv2.CAP_PROP_FRAME_COUNT))

    # Face detector
    face_detector = dlib.get_frontal_face_detector()

    # Load model
    model1, *_ = model_selection(modelname='densenet121', num_out_classes=2)
    if model_path is not None:
        model1 = torch.load(model_path)
    if cuda:
        model1 = model1.cuda()

    model2, *_ = model_selection(modelname='resnet18', num_out_classes=2)
    if model_path is not None:
        model2 = torch.load(model_path)
    if cuda:
        model2 = model2.cuda()

    model3, *_ = model_selection(modelname='xception', num_out_classes=2)
    if model_path is not None:
        model3 = torch.load(model_path)
    if cuda:
        model3 = model3.cuda()

    # Frame numbers and length of output video
    frame_num = 0
    assert start_frame < num_frames - 1
    end_frame = end_frame if end_frame else num_frames

    predictions1 = []
    predictions2 = []
    predictions3 = []

    while reader.isOpened():
        _, image = reader.read()
        if image is None:
            break
        frame_num += 1

        if frame_num < start_frame:
            continue

        # Image size
        height, width = image.shape[:2]

        # 2. Detect with dlib
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        faces = face_detector(gray, 1)
        if len(faces):
            # For now only take biggest face
            face = faces[0]

            # --- Prediction ---------------------------------------------------
            # Face crop with dlib and bounding box scale enlargement
            x, y, size = get_boundingbox(face, width, height)
            cropped_face = image[y:y+size, x:x+size]

            # Actual prediction using our model
            # Model 1 : densenet121
            prediction, output1 = predict_with_model(cropped_face, model1,
                                                    cuda=cuda)
            predictions1.append(prediction)

            # Model 2 : resnet18
            prediction2, output2 = predict_with_model(cropped_face, model2,
                                                    cuda=cuda)
            predictions2.append(prediction2)

            # Model 3 : xception
            prediction3, output3 = predict_with_model(cropped_face, model3,
                                                    cuda=cuda)
            predictions3.append(prediction3)
            # ------------------------------------------------------------------

        if frame_num >= end_frame:
            break

    # return predictions but wich number appear the most
    model1_prediction = max(set(predictions1), key=predictions1.count)
    model2_prediction = max(set(predictions2), key=predictions2.count)
    model3_prediction = max(set(predictions3), key=predictions3.count)
    print(model1_prediction, model2_prediction, model3_prediction)
    final = [model1_prediction, model2_prediction, model3_prediction]

    return max(set(final), key=final.count)

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
already = {'df2_fake_id0_id16_0007.mp4': '[1, 1, 0, 1]', 'df2_fake_id30_id6_0008.mp4': '[1, 1, 0, 1]', 'df2_fake_id0_id20_0007.mp4': '[0, 1, 1, 1]', 'df2_fake_id31_id20_0009.mp4': '[0, 1, 0, 0]', 'df2_fake_id0_id3_0007.mp4': '[0, 1, 0, 0]', 'df2_fake_id31_id29_0002.mp4': '[0, 0, 0, 0]', 'df2_fake_id10_id12_0007.mp4': '[0, 0, 1, 0]', 'df2_fake_id35_id16_0007.mp4': '[0, 0, 0, 0]', 'df2_fake_id16_id3_0006.mp4': '[0, 1, 1, 1]', 'df2_fake_id35_id17_0008.mp4': '[1, 0, 1, 1]', 'df2_fake_id16_id9_0003.mp4': '[1, 1, 0, 1]', 'df2_fake_id35_id20_0003.mp4': '[0, 0, 0, 0]', 'df2_fake_id17_id26_0003.mp4': '[0, 1, 0, 0]', 'df2_fake_id35_id20_0007.mp4': '[1, 0, 1, 1]', 'df2_fake_id17_id28_0005.mp4': '[0, 1, 0, 0]', 'df2_fake_id35_id33_0004.mp4': '[1, 0, 0, 0]', 'df2_fake_id17_id3_0005.mp4': '[1, 1, 0, 1]', 'df2_fake_id35_id6_0000.mp4': '[1, 1, 1, 1]', 'df2_fake_id1_id31_0000.mp4': '[0, 0, 1, 0]', 'df2_fake_id35_id9_0009.mp4': '[0, 0, 0, 0]', 'df2_fake_id1_id3_0005.mp4': '[1, 1, 0, 1]', 'df2_fake_id37_id17_0000.mp4': '[0, 1, 1, 1]', 'df2_fake_id20_id16_0005.mp4': '[1, 1, 1, 1]', 'df2_fake_id37_id31_0006.mp4': '[1, 0, 1, 1]', 'df2_fake_id20_id1_0006.mp4': '[1, 0, 1, 1]', 'df2_fake_id38_id28_0002.mp4': '[0, 1, 0, 0]', 'df2_fake_id20_id2_0008.mp4': '[0, 1, 1, 1]', 'df2_fake_id38_id28_0003.mp4': '[0, 0, 0, 0]', 'df2_fake_id20_id31_0002.mp4': '[1, 1, 0, 1]', 'df2_fake_id38_id30_0003.mp4': '[1, 1, 1, 1]', 'df2_fake_id20_id32_0005.mp4': '[0, 1, 1, 1]', 'df2_fake_id3_id0_0008.mp4': '[0, 0, 0, 0]', 'df2_fake_id21_id23_0002.mp4': '[1, 1, 1, 1]', 'df2_fake_id3_id30_0007.mp4': '[0, 1, 1, 1]', 'df2_fake_id21_id27_0002.mp4': '[1, 0, 0, 0]', 'df2_fake_id3_id31_0000.mp4': '[0, 0, 1, 0]', 'df2_fake_id22_id28_0005.mp4': '[0, 1, 1, 1]', 'df2_fake_id40_id44_0002.mp4': '[1, 1, 0, 1]', 'df2_fake_id23_id20_0009.mp4': '[1, 1, 0, 1]', 'df2_fake_id41_id40_0004.mp4': '[1, 0, 1, 1]', 'df2_fake_id23_id25_0006.mp4': '[1, 1, 0, 1]', 'df2_fake_id41_id40_0006.mp4': '[1, 0, 1, 1]', 'df2_fake_id23_id35_0009.mp4': '[0, 1, 1, 1]', 'df2_fake_id41_id46_0009.mp4': '[1, 1, 0, 1]', 'df2_fake_id23_id38_0003.mp4': '[0, 1, 0, 0]', 'df2_fake_id41_id48_0003.mp4': '[1, 0, 1, 1]', 'df2_fake_id23_id3_0009.mp4': '[1, 0, 1, 1]', 'df2_fake_id46_id42_0006.mp4': '[0, 1, 0, 0]', 'df2_fake_id24_id25_0002.mp4': '[1, 0, 1, 1]', 'df2_fake_id46_id47_0002.mp4': '[0, 0, 0, 0]', 'df2_fake_id24_id26_0003.mp4': '[1, 1, 1, 1]', 'df2_fake_id48_id46_0005.mp4': '[0, 1, 0, 0]', 'df2_fake_id26_id1_0002.mp4': '[1, 1, 0, 1]', 'df2_fake_id48_id47_0003.mp4': '[0, 0, 0, 0]', 'df2_fake_id26_id38_0004.mp4': '[0, 1, 1, 1]', 'df2_fake_id49_id51_0006.mp4': '[1, 0, 0, 0]', 'df2_fake_id28_id22_0005.mp4': '[0, 1, 1, 1]', 'df2_fake_id49_id53_0008.mp4': '[0, 0, 1, 0]', 'df2_fake_id28_id38_0007.mp4': '[0, 1, 0, 0]', 'df2_fake_id4_id20_0005.mp4': '[1, 0, 0, 0]', 'df2_fake_id28_id9_0003.mp4': '[1, 0, 0, 0]', 'df2_fake_id4_id26_0007.mp4': '[0, 1, 1, 1]', 'df2_fake_id28_id9_0004.mp4': '[0, 1, 1, 1]', 'df2_fake_id4_id30_0001.mp4': '[0, 1, 1, 1]', 'df2_fake_id29_id28_0006.mp4': '[0, 1, 0, 0]', 'df2_fake_id4_id3_0003.mp4': '[1, 1, 1, 1]', 'df2_fake_id29_id34_0004.mp4': '[0, 1, 0, 0]', 'df2_fake_id4_id9_0006.mp4': '[1, 1, 1, 1]', 'df2_fake_id29_id38_0001.mp4': '[0, 1, 0, 0]', 'df2_fake_id50_id52_0001.mp4': '[0, 0, 0, 0]', 'df2_fake_id2_id35_0000.mp4': '[0, 1, 0, 0]', 'df2_fake_id51_id54_0007.mp4': '[0, 1, 1, 1]', 'df2_fake_id2_id35_0007.mp4': '[1, 0, 1, 1]', 'df2_fake_id52_id50_0007.mp4': '[0, 0, 0, 0]', 'df2_fake_id2_id3_0005.mp4': '[1, 0, 1, 1]', 'df2_fake_id52_id53_0006.mp4': '[0, 0, 1, 0]', 'df2_fake_id30_id20_0004.mp4': '[0, 0, 0, 0]', 'df2_fake_id53_id54_0006.mp4': '[0, 0, 0, 0]', 'df2_fake_id30_id23_0002.mp4': '[1, 0, 0, 0]', 'df2_fake_id53_id57_0007.mp4': '[1, 0, 0, 0]', 'df2_fake_id30_id33_0008.mp4': '[0, 0, 1, 0]', 'df2_fake_id31_id2_0005.mp4': '[0, 0, 1, 0]', 'df2_fake_id54_id49_0002.mp4': '[0, 0, 0, 0]', 'df2_fake_id31_id9_0004.mp4': '[1, 1, 1, 1]', 'df2_fake_id54_id55_0006.mp4': '[0, 0, 1, 0]', 'df2_fake_id33_id20_0005.mp4': '[1, 1, 0, 1]', 'df2_fake_id55_id53_0009.mp4': '[1, 1, 1, 1]', 'df2_fake_id33_id37_0000.mp4': '[1, 1, 0, 1]'}

In [ ]:
import os
from os.path import join
import time

output_file_path = "/content/drive/MyDrive/output22.txt"  # Change the path as needed
video_path = "/content/drive/MyDrive/lucky2"

with open(output_file_path, 'w') as output_file:
    if video_path.endswith('.mp4') or video_path.endswith('.avi'):
        prediction = test_full_image_network(video_path=video_path)
        output_file.write('Video: {}, Prediction: {}\n'.format(video_path, prediction))
    else:
      output_file = open(output_file_path, 'w')
      videos = os.listdir(video_path)
      predictions = {}

      for video in videos:
          if video in already:
            print("Already")
            continue
          video_paths = join(video_path, video)
          start_time = time.time()
          prediction = test_full_image_network(video_path=video_paths, model_path=None)
          print("--- %s seconds ---" % (time.time() - start_time))
          predictions[video] = prediction
          output_file.write('Video: {}, Prediction: {}\n'.format(video, prediction))
          print('Video: {}, Prediction: {}'.format(video, prediction))

      output_file.close()

      print(predictions)